In [1]:
from salt import Salty
import pandas as pd
import numpy as np
from gnews import NewsParser
from tqdm import tqdm
from starwars import Darth

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Couldn't initialize gpu


In [2]:
path = "../datasets/"
imdb = pd.read_csv(path + "IMDB Dataset.csv")
fina = pd.read_csv(path + "financial-headlines.csv")
data = pd.concat([imdb, fina])  # combine into big
salty = Salty()
#salty.bundle_train_test(data)
salty.load_model('../models/latest-model')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1934, 128)         256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 1934, 128)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [3]:
news = NewsParser()
art = news.get_articles('coinbase')

In [4]:
X = salty.create_X(imdb['review'].values[:20])
X = salty.create_X(art['text'].values)

In [5]:
m = []
for i in range(len(X)):
    #print(art['text'].values[i])
    n = salty.predict_next_value(X[i])
    if n==0:
        n=-1
    m.append(n)
total_avg = np.average(m)
print(f"Avg sentiment score: {total_avg}")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmps2dyr29y.py, line 13)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmps2dyr29y.py, line 13)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpqi5kngv5.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Li

In [6]:
d = Darth()
d.sentiment(['robinhood'])

{'robinhood': {'avg': 0.28133254089329607, 'nice': 'positive'}}